# <p style="text-align: center;"> Final Project - Introduction to Data Science</p>
# <p style="text-align: center;"> <b> Data Collecting </b></p>
---

## Member Information
| Name              | ID       |
|-------------------|----------|
|Tran Đinh Quang    | 21127406 |
|Nguyen Hong Hanh   | 21127503 |
|Do Quoc Tri  | 21127556 |
| Nguyen Khanh Nhan | 21127657         |


## Table of contents



---

## Import libaries

In [1]:
import requests
import sys
import json
import pandas as pd
from datetime import datetime, timedelta, date

Dữ liệu được lấy về từ trang web [Visualcrossing](https://www.visualcrossing.com/). Dữ liệu trong bài là dữ liệu về thời tiết của thành phố Hồ Chí Minh từ  01/01/2007 đến 31/12/2022. Do mã id chi cho phép lấy về 1000 dòng dữ liệu mỗi ngày vì vậy mỗi lần chạy lẽ lấy về 1000 dòng dữ liệu và ghi lại ngày đã chạy vào file để tiếp tục vào các lần chạy sau.

In [2]:
key = 'PEVKTTBLZUTL89Z6WHQ5V2HRK'
num_of_days = 999

with open('../data/lastdate.txt', 'r') as file:
  start = file.read()

end = datetime.strptime(start, '%Y-%m-%d') + timedelta(days=num_of_days)
end = end.strftime('%Y-%m-%d')

In [ ]:
response = requests.request("GET", f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Ho%20Chi%20Minh/{start}/{end}?unitGroup=us&include=days&key={key}&contentType=json")
if response.status_code!=200:
  print('Unexpected Status code: ', response.status_code)
  sys.exit()  


# Parse the results as JSON
jsonData = response.json()

In [4]:
location_info ={}
features = ['latitude', 'longitude', 'resolvedAddress', 'address', 'timezone', 'tzoffset']
for f in features:
  location_info[f] = jsonData[f]
location_info

{'latitude': 10.7764,
 'longitude': 106.701,
 'resolvedAddress': 'Quận 1, Hồ Chí Minh, Việt Nam',
 'address': 'Ho Chi Minh',
 'timezone': 'Asia/Ho_Chi_Minh',
 'tzoffset': 7.0}

In [9]:
df = pd.DataFrame.from_dict(jsonData['days'])
df.head()

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,sunrise,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source
0,2009-09-27,1253984400,87.9,73.5,78.9,100.9,73.5,82.5,76.5,93.0,...,05:42:03,1254004923,17:46:17,1254048377,0.27,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,[48900099999],obs
1,2009-09-28,1254070800,84.8,75.3,79.6,94.0,75.3,82.7,75.7,88.4,...,05:42:00,1254091320,17:45:39,1254134739,0.30,Overcast,Cloudy skies throughout the day.,cloudy,[48900099999],obs
2,2009-09-29,1254157200,84.5,77.1,80.5,93.5,77.1,84.6,76.2,87.0,...,05:41:58,1254177718,17:45:01,1254221101,0.34,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,[48900099999],obs
3,2009-09-30,1254243600,87.9,74.6,79.3,100.2,74.6,82.4,76.3,91.7,...,05:41:56,1254264116,17:44:24,1254307464,0.37,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,[48900099999],obs
4,2009-10-01,1254330000,89.7,75.3,80.9,100.5,75.3,85.4,76.3,87.2,...,05:41:54,1254350514,17:43:46,1254393826,0.40,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,[48900099999],obs


In [14]:
df.to_csv(f'../data/raw_{start}_{end}.csv', index=False)

**Caution** Do not run next cell before finished

In [13]:
end = datetime.strptime(end, '%Y-%m-%d') + timedelta(days=1)
end = end.strftime('%Y-%m-%d')
with open('../data/lastdate.txt', 'w') as file:
  file.write(end)